# Open an Existing Project

Use existing example: ```C:\Users\Public\Documents\Vector CANape 17\Examples\XCPDemo``` for 'developing'.

In [1]:
import ctypes, os

# Install CANape. Not included for copyright reasons.
header_file = r"C:\Program Files\Vector CANape 17\CANapeAPI\CANapAPI.h"
assert os.path.exists(header_file)
CANapAPI_dll= r"C:\Program Files\Vector CANape 17\CANapeAPI\CANapAPI64.dll"
assert os.path.exists(CANapAPI_dll)

In [2]:
# Load the library and assign it to dll.
dll = ctypes.windll.LoadLibrary(CANapAPI_dll)

class struct_tAsap3Hdl(ctypes.Structure):
    pass
TAsap3Hdl = ctypes.POINTER(struct_tAsap3Hdl)

TRecorderID = ctypes.POINTER(ctypes.c_ulong)
TModulHdl = ctypes.c_ushort
TScriptHdl = ctypes.c_ulong
TTime = ctypes.c_ulong

In [3]:
handle=TAsap3Hdl()
handle_p = ctypes.byref(handle)

# CANape requires absolute path.
workingDir = os.path.abspath(r"C:\Users\Public\Documents\Vector CANape 17\Examples\XCPDemo")
# Maximum response time (ms)
responseTimeout = 10000 # 10 seconds
fifoSize = 8192
sampleSize = 256
debugMode = True
# Clear CANape device list.
clearDeviceList = True
# Start CANape in hex mode
bHexmode = False
# Start CANape in modal mode. (NonModal = True)
bModalMode = True

dll.Asap3Init5.restype = ctypes.c_bool
dll.Asap3Init5.argtypes = (
    ctypes.POINTER(TAsap3Hdl),
    ctypes.c_ulong,
    ctypes.c_char_p,
    ctypes.c_ulong,
    ctypes.c_ulong,
    ctypes.c_bool,
    ctypes.c_bool,
    ctypes.c_bool,
    ctypes.c_bool
)

In [4]:
result = dll.Asap3Init5(
    handle_p,
    responseTimeout,
    workingDir.encode("UTF-8"),
    fifoSize,
    sampleSize,
    debugMode,
    clearDeviceList,
    bHexmode,
    bModalMode,
)


In [5]:
dll.Asap3GetInteractiveMode.argtypes=(TAsap3Hdl, ctypes.POINTER(ctypes.c_bool))
dll.Asap3GetInteractiveMode.restype=ctypes.c_bool

In [6]:
mode = ctypes.c_bool(0)
result = dll.Asap3GetInteractiveMode(handle, ctypes.byref(mode))
result, mode

(True, c_bool(False))

In [7]:
dll.Asap3SetInteractiveMode.argtypes=(TAsap3Hdl, ctypes.c_bool)
dll.Asap3SetInteractiveMode.restype=ctypes.c_bool

In [8]:
dll.Asap3SetInteractiveMode(handle, True)

True

In [9]:
mode = ctypes.c_bool(0)
result = dll.Asap3GetInteractiveMode(handle, ctypes.byref(mode))
result, mode

(True, c_bool(True))

In [10]:
dll.Asap3SetInteractiveMode(handle, False)

True

In [11]:
dll.Asap3GetModuleCount.argtypes=(TAsap3Hdl, ctypes.POINTER(ctypes.c_ulong))
dll.Asap3SetInteractiveMode.restype=ctypes.c_bool

In [12]:
count = ctypes.c_ulong()
result = dll.Asap3GetModuleCount(handle, ctypes.byref(count))

In [13]:
result, count

(1, c_ulong(1))

In [14]:
class TModulHdl(ctypes.c_ushort):
    def __init__(self, value=0):
        self.value=value
    def __repr__(self):
        return f"Module<{self.value}>"

In [38]:
module = TModulHdl(0)

dll.Asap3GetModuleHandle.argtypes=(TAsap3Hdl, ctypes.c_char_p, ctypes.POINTER(TModulHdl))
dll.Asap3GetModuleHandle.restype=ctypes.c_bool

dll.Asap3GetModuleName.argtypes=(TAsap3Hdl, TModulHdl, ctypes.POINTER(ctypes.c_char_p));
dll.Asap3GetModuleName.restype=ctypes.c_bool

name = ctypes.c_char_p()
dll.Asap3GetModuleName(handle, module, ctypes.byref(name))

True

In [23]:
name

c_char_p(1675370474528)

In [24]:
name.value

b'XCPsim'

In [39]:
mdl = TModulHdl()
result = dll.Asap3GetModuleHandle(handle, name.value, ctypes.byref(mdl))
result, mdl

(True, Module<0>)

In [43]:
class TModulHdl(ctypes.c_ushort):
    def __init__(self, value=0, handle=None):
        self.value=value
        self.handle=handle
    def __repr__(self):
        return f"Module<{self.value}>"
    @property
    def name(self):
        name = ctypes.c_char_p()
        result = dll.Asap3GetModuleName(self.handle, self.value,  ctypes.byref(name))
        assert result==True
        return name.value.decode("UTF-8")
        
        

In [44]:
mdl = TModulHdl(0, handle)

In [45]:
mdl.name

'XCPsim'